# Paralelismo a Nível de Dados (Data Parallelism)

O paralelismo a nível de dados, ou "Data Parallelism", refere-se à técnica de dividir um grande conjunto de dados em partes menores e processar essas partes em paralelo. No contexto de aprendizado de máquina, isso é frequentemente usado para treinar modelos em várias GPUs ou máquinas, onde cada GPU/máquina processa um subconjunto dos dados.

Vamos usar o PyTorch para demonstrar o paralelismo a nível de dados. O PyTorch oferece uma classe `DataParallel` que facilita o treinamento de modelos em várias GPUs.

### Exemplo: Treinamento de um Modelo Usando Data Parallelism com PyTorch

```python
import torch
import torch.nn as nn
import torch.optim as optim

# Verificar a disponibilidade de GPUs
num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")

# Gerar dados sintéticos
n_samples = 10000
X = torch.rand(n_samples, 1) * 10 - 5  # Valores entre -5 e 5
y = X * 3 + torch.randn(n_samples, 1) * 0.5  # Linha reta com algum ruído

# Modelo de rede neural simples
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layer = nn.Linear(1, 1)
    
    def forward(self, x):
        return self.layer(x)

model = SimpleNN()

# Se houver mais de uma GPU, use DataParallel
if num_gpus > 1:
    model = nn.DataParallel(model)

model = model.cuda()  # Mova o modelo para a GPU
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Treinamento
num_epochs = 500
for epoch in range(num_epochs):
    # Transferir dados para a GPU
    inputs, labels = X.cuda(), y.cuda()
    
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print("Training complete!")
```

Neste exemplo:

- Verificamos a disponibilidade de GPUs usando `torch.cuda.device_count()`.
- Geramos um conjunto de dados sintético.
- Criamos um modelo simples de rede neural com uma única camada linear.
- Se houver mais de uma GPU disponível, usamos `nn.DataParallel` para envolver o modelo, permitindo que ele seja treinado em várias GPUs.
- Treinamos o modelo na GPU (ou GPUs).

Ao executar este código em um Jupyter Notebook com acesso a várias GPUs, o PyTorch dividirá automaticamente o mini-lote de entrada em várias GPUs e agregará os gradientes antes de atualizar os pesos do modelo.

Lembre-se de que, para aproveitar ao máximo o paralelismo a nível de dados, você também deve usar carregadores de dados (`DataLoader`) com tamanhos de lote apropriados e talvez aumentar o tamanho do lote à medida que adiciona mais GPUs.